In [426]:
# 操作 browser 的 驅動程式
from selenium import webdriver

# 負責開啟和關閉 Chrome 的套件
from selenium.webdriver.chrome.service import Service

# 自動下載 Chrome Driver 的套件
from webdriver_manager.chrome import ChromeDriverManager

# 啟動瀏覽器工具的選項
from selenium.webdriver.chrome.options import Options

# 例外處理的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待、了解某個元素的狀態，通常與 exptected_conditions 和 By 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 隨機取得 User-Agent
from fake_useragent import UserAgent
# ua = UserAgent()

import pymysql
from pymysql.err import IntegrityError
from time import sleep
import json
import random
import os
from datetime import datetime
from datetime import timedelta
import re
from pprint import pprint


In [427]:
user_agent_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393",
    "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36"
    "Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
    # Add more user agent strings as needed
]
user_agent = random.choice(user_agent_list)

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false')  # 不載入圖
my_options.add_argument(f'--user-agent={user_agent}')  # 隨機 user-angent 
my_options.add_experimental_option(
    "excludeSwitches", ['enable-automation', 'enable-logging'])  # 沒有異常log


driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install()))

In [428]:
# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'crawler_data')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})

In [429]:
# url = 'https://www.cmoney.tw/forum/popular/buzz'
url = 'https://www.cmoney.tw/forum/official/359804'
listData = []
results = []

In [430]:
def visit():
    driver.get(url)


def filterFunc():
    try:
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, '#__layout > div > div > div.cm-blackbar__body.cm-blackbar__body-normal > div.forum__body > div > div > div > div.page__article > section:nth-child(1) > div > nav > ul > li:nth-child(3) > button > div')
            )
        )
        driver.find_element(
            By.CSS_SELECTOR, "#__layout > div > div > div.cm-blackbar__body.cm-blackbar__body-normal > div.forum__body > div > div > div > div.page__article > section:nth-child(1) > div > nav > ul > li:nth-child(3) > button > div"
        ).click()

        sleep(2)

    except TimeoutException:
        print("等待逾時")

def scroll():

    innerHeight = 0
    offset = 0
    count = 0
    limit = 3

    while count <= limit:
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')

        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(3)

        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )

        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1

        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if offset >= 1000:
            break

def parse():
    global listData
    # listData.clear()

    elements = driver.find_elements(
    By.CSS_SELECTOR,
    'section > article.articleContent'
    )

    # extract data
    for elm in elements:
        listData.clear()

        try:
            # stock name from KOL recommand
            stocks = elm.find_element(By.CSS_SELECTOR, "ul.articleTags__list").get_attribute('innerText')
            stock_list = stocks.split('\n')

            if stock_list[0] == '':
                continue
            
            # title
            title = elm.find_element(By.CSS_SELECTOR, "h3.articleContent__title.text-white-800 > a").get_attribute('innerText')
            # title_name = title.get_attribute('innerText')

            # post_type
            post_type = title[1:3]

            # poster
            poster = elm.find_element(By.CSS_SELECTOR, "div.member__name.text-white-800").get_attribute('innerText')
            # poster_name = poster.get_attribute('innerText')


            # time
            time = elm.find_element(By.CSS_SELECTOR, "div.member__remark > a").get_attribute('innerText')
            
            # actual post-time

            loc_dt = datetime.today() 


            re_time = r'^\d+'
            if "分鐘前" in time:
                # loc_dt = datetime.today() 
                match_time = re.match(re_time, time)
                time_del = timedelta(minutes=int(match_time[0]))
                post_time = loc_dt - time_del

                # print(match_time[0])
                # print(time_del)

            elif "小時前" in time:
                # loc_dt = datetime.today() 
                match_hour = re.match(re_time, time)
                time_del = timedelta(hours=int(match_hour[0]))
                post_time = loc_dt - time_del

                # print(match_hour[0])
                # print(loc_dt)
                # print(time_del)
                # print(loc_dt - time_del)
            
            elif "昨天" in time:
                time = time.split(" ")[1]
                time_hour = time.split(":")[0]
                time_mins = time.split(":")[1]

                # print('time_hour', time_hour)
                # print('time_mins', time_mins)

                post_time = loc_dt.replace(hour=int(time_hour), minute=int(time_mins), second=0, microsecond=0)
                post_time = post_time - timedelta(days=1)
                
            result = {
                'title': title,
                'post_type': post_type,
                'poster' : poster,
                'stocks': stock_list,
                'time' : time,
                'post_time' : post_time.strftime("%Y/%m/%d %H:%M:%S")
            }

            # # 新增進資料庫
            # connection = pymysql.connect(host='localhost', port=3306, user='sunny', password='sunny', database='stock_news')
            # cursor = connection.cursor()
            # sql = 'INSERT INTO ss_news (title, post_type, poster, stocks, time, post_time) VALUES ("%s", "%s", "%s", "%s", "%s", "%s");'\
            #                     % (title, post_type, poster, stock_list, time, post_time.strftime("%Y/%m/%d %H:%M:%S"))
            # cursor.execute(sql)
            # connection.commit()
            # connection.close()

        # except IntegrityError as e:
        #     # 如果出現IntegrityError，也就是PK重複
        #     print('已經抓取到最新資料')
        #     connection.close()
        #     print('關閉資料庫連接')
        #     # lock = 1
        #     break

        except:
            continue
        
        # print("=="*20)
        results.append(result)


def check_crawler_time():

    loc_dt = datetime.today() 
    
    # initial last_time
    last_time = driver.find_elements(
    By.CSS_SELECTOR,
    'div > a.link.member__period.text-white-600'
    )

    while "昨天" not in last_time[-1].text:
        
        scroll()

        # update last_time
        last_time = driver.find_elements(
        By.CSS_SELECTOR,
        'div > a.link.member__period.text-white-600'
        )

    # initial post_time
    time = last_time[-1].text.split(" ")[1]
    time_hour = time.split(":")[0]
    time_mins = time.split(":")[1]

    post_time = loc_dt.replace(hour=int(time_hour), minute=int(time_mins), second=0, microsecond=0)
    post_time = post_time - timedelta(days=1)

    yesterday9pm = loc_dt.replace(hour=21, minute=0, second=0, microsecond=0)
    yesterday9pm = yesterday9pm - timedelta(days=1)

    while post_time >= yesterday9pm:

        scroll()

        # update post_time
        last_time = driver.find_elements(
        By.CSS_SELECTOR,
        'div > a.link.member__period.text-white-600'
        )

        time = last_time[-1].text.split(" ")[1]
        time_hour = time.split(":")[0]
        time_mins = time.split(":")[1]

        post_time = loc_dt.replace(hour=int(time_hour), minute=int(time_mins), second=0, microsecond=0)
        post_time = post_time - timedelta(days=1)

    


In [431]:
if __name__ == '__main__':
    visit()
    check_crawler_time()
    parse()
    

In [434]:
# listData
results
# len(results)

[{'title': '[情報] 2/21全市場當沖虧損最多的股票',
  'post_type': '情報',
  'poster': '批踢踢實業坊 Stock 板',
  'stocks': ['虹堡', '營邦', '東哥遊艇', '威盛'],
  'time': '20分鐘前',
  'post_time': '2023/02/21 22:04:14'},
 {'title': '[情報] 112年02月21日信用交易統計',
  'post_type': '情報',
  'poster': '批踢踢實業坊 Stock 板',
  'stocks': ['華航', '中興電', '玉山金', '台積電', '聯電'],
  'time': '20分鐘前',
  'post_time': '2023/02/21 22:04:14'},
 {'title': '[情報] 6770 力積電 111年下半年股利分派暨除息基準日',
  'post_type': '情報',
  'poster': '批踢踢實業坊 Stock 板',
  'stocks': ['力積電'],
  'time': '20分鐘前',
  'post_time': '2023/02/21 22:04:14'},
 {'title': '[情報] 3037欣興第四季財報',
  'post_type': '情報',
  'poster': '批踢踢實業坊 Stock 板',
  'stocks': ['欣興', '景碩'],
  'time': '53分鐘前',
  'post_time': '2023/02/21 21:31:14'},
 {'title': '[情報] 3037 欣興 111年股利分派',
  'post_type': '情報',
  'poster': '批踢踢實業坊 Stock 板',
  'stocks': ['欣興'],
  'time': '1小時前',
  'post_time': '2023/02/21 21:24:14'},
 {'title': '[新聞] 臺慶科車用添新動能 取得陸系電動車電',
  'post_type': '新聞',
  'poster': '批踢踢實業坊 Stock 板',
  'stocks': ['臺慶科'],
  'time